# Advanced Analysis 3: Process Variant Clustering - Outcome-Based Pattern Discovery

## 1. Hypotheses

### Research Questions:
- Do processes leading to different outcomes (Denied, Cancelled, Pending) have different activity patterns?
- Can we identify distinct clusters of process variants that are associated with specific outcomes?
- Are there perceivable differences in processes that lead to certain outcomes?
- Which activity patterns characterize each outcome type?
- Can we predict case outcomes based on process variant clusters?

### Why this matters for prediction/simulation:
- Understanding if different outcomes follow distinct process patterns enables more accurate outcome prediction
- Clusters can represent different process scenarios in simulation models, each with outcome-specific probabilities
- Identifying outcome-specific patterns helps build targeted prediction models for each outcome type
- Early detection of outcome-specific patterns enables proactive intervention
- Process redesign can focus on shifting cases from negative outcome clusters to positive ones


In [ ]:
import pm4py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.stats import chi2_contingency
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 10

# Create output directory
output_dir = Path("../../Results/Advanced_Analysis/variant_clustering")
output_dir.mkdir(parents=True, exist_ok=True)

print("Libraries imported successfully")


Libraries imported successfully


In [7]:
# Load event log
log_path = "../../Dataset/BPI Challenge 2017.xes"
log = pm4py.read_xes(log_path)
df = pm4py.convert_to_dataframe(log)

# Ensure timestamp is datetime
df['time:timestamp'] = pd.to_datetime(df['time:timestamp'], utc=True)

# Sort by case and timestamp
df = df.sort_values(['case:concept:name', 'time:timestamp'])

print(f"Loaded {len(df):,} events")
print(f"Number of cases: {df['case:concept:name'].nunique():,}")
print(f"Number of unique activities: {df['concept:name'].nunique()}")
print(f"Number of unique resources: {df['org:resource'].nunique():,}")


Loaded 1,202,267 events
Number of cases: 31,509
Number of unique activities: 26
Number of unique resources: 149


## 2. Approach and Results

### Methodology:
- Extract process variants (activity sequences) from the event log
- Classify case outcomes based on whether endpoint activities appear anywhere in the trace
- Create feature vectors for variants:
  - Activity presence/absence (binary features)
  - Normalized activity frequencies
- Apply dimensionality reduction (PCA/t-SNE) for visualization and noise reduction
- Perform clustering (K-means) with optimal cluster selection using silhouette score
- Analyze cluster-outcome relationships:
  - Visualize clusters colored by outcome to see if outcomes separate naturally
  - Create cross-tabulations and heatmaps showing cluster-outcome associations
  - Calculate cluster purity metrics for each outcome
  - Apply statistical tests (chi-square) to quantify cluster-outcome associations
- Compare activity patterns across outcome groups and clusters
- Analyze performance metrics (duration, resource usage) by cluster and outcome


In [8]:
# Extract variants and case-level information
case_data = df.groupby('case:concept:name').agg({
    'time:timestamp': ['min', 'max'],
    'concept:name': lambda x: list(x),  # Activity sequence
    'org:resource': lambda x: list(x),  # Resource sequence
    'case:LoanGoal': 'first',
    'case:ApplicationType': 'first',
    'case:RequestedAmount': 'first'
}).reset_index()

case_data.columns = ['case_id', 'start_time', 'end_time', 'activity_sequence', 
                     'resource_sequence', 'loan_goal', 'app_type', 'requested_amount']

# Calculate case duration
case_data['duration_days'] = (case_data['end_time'] - case_data['start_time']).dt.total_seconds() / (24 * 3600)

# Classify outcomes based on whether endpoint activities appear anywhere in the trace
# Priority: Denied > Cancelled > Pending (if multiple endpoints appear)
def classify_outcome(activities):
    # Check if activities is None or empty
    if activities is None:
        return 'Unknown'
    try:
        if len(activities) == 0:
            return 'Unknown'
    except (TypeError, AttributeError):
        return 'Unknown'
    
    # Check if any endpoint activity appears in the trace
    activities_set = set(activities)
    
    if 'A_Denied' in activities_set:
        return 'Denied'
    if 'A_Cancelled' in activities_set:
        return 'Cancelled'
    if 'A_Pending' in activities_set:
        return 'Pending'
    return 'Other'

case_data['outcome'] = case_data['activity_sequence'].apply(classify_outcome)

# Create variant string for grouping
case_data['variant_string'] = case_data['activity_sequence'].apply(lambda x: ' -> '.join(str(a) for a in x))

print(f"Prepared {len(case_data):,} cases")
print(f"Number of unique variants: {case_data['variant_string'].nunique():,}")
print(f"\nOutcome distribution:")
print(case_data['outcome'].value_counts())


Prepared 31,509 cases
Number of unique variants: 15,930

Outcome distribution:
outcome
Pending      17228
Cancelled    10431
Denied        3752
Other           98
Name: count, dtype: int64


In [9]:
# Get all unique activities
all_activities = set()
for seq in case_data['activity_sequence']:
    all_activities.update(seq)
all_activities = sorted(list(all_activities))

print(f"Total unique activities: {len(all_activities)}")

# Create feature vectors: activity presence/absence and frequencies
variant_features = []

for idx, row in case_data.iterrows():
    seq = row['activity_sequence']
    
    # Binary features: activity presence
    activity_presence = [1 if activity in seq else 0 for activity in all_activities]
    
    # Frequency features: how many times each activity appears
    activity_freq = [seq.count(activity) for activity in all_activities]
    
    # Normalized frequency
    total_activities = len(seq)
    activity_freq_norm = [freq / total_activities if total_activities > 0 else 0 for freq in activity_freq]
    
    # Combine features
    features = activity_presence + activity_freq_norm
    variant_features.append(features)

# Convert to numpy array
X = np.array(variant_features)
print(f"Feature matrix shape: {X.shape}")

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Features standardized")


Total unique activities: 26
Feature matrix shape: (31509, 52)
Features standardized


In [10]:
# Apply PCA for dimensionality reduction (for visualization and to reduce noise)
pca = PCA(n_components=50)  # Keep 50 components to capture most variance
X_pca = pca.fit_transform(X_scaled)

print(f"PCA explained variance ratio (first 10 components): {pca.explained_variance_ratio_[:10]}")
print(f"PCA cumulative explained variance (first 10): {np.cumsum(pca.explained_variance_ratio_[:10])}")
print(f"PCA total explained variance (50 components): {np.sum(pca.explained_variance_ratio_):.4f}")

# Apply t-SNE for 2D visualization
print("\nComputing t-SNE (this may take a while)...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
X_tsne = tsne.fit_transform(X_pca[:, :30])  # Use first 30 PCA components for t-SNE
print("t-SNE completed")


PCA explained variance ratio (first 10 components): [0.37825921 0.11144788 0.08659109 0.06899701 0.0598742  0.04755799
 0.04385306 0.04123079 0.03865201 0.03684944]
PCA cumulative explained variance (first 10): [0.37825921 0.48970708 0.57629818 0.64529518 0.70516938 0.75272737
 0.79658044 0.83781123 0.87646323 0.91331267]
PCA total explained variance (50 components): 1.0000

Computing t-SNE (this may take a while)...


TypeError: TSNE.__init__() got an unexpected keyword argument 'n_iter'

In [ ]:
# Visualize t-SNE colored by outcome to see if outcomes separate naturally
fig, ax = plt.subplots(figsize=(16, 10))

outcome_colors = {'Denied': 'red', 'Cancelled': 'orange', 'Pending': 'green', 'Other': 'gray', 'Unknown': 'black'}
for outcome in case_data['outcome'].unique():
    mask = case_data['outcome'] == outcome
    ax.scatter(X_tsne[mask, 0], X_tsne[mask, 1], 
              c=outcome_colors.get(outcome, 'gray'), label=outcome, 
              alpha=0.6, s=10, edgecolors='black', linewidth=0.1)

ax.set_xlabel('t-SNE Component 1', fontsize=12)
ax.set_ylabel('t-SNE Component 2', fontsize=12)
ax.set_title('Process Variants in t-SNE Space (Colored by Outcome)', 
            fontsize=14, fontweight='bold')
ax.legend(title='Outcome', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(output_dir / 'tsne_colored_by_outcome.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: tsne_colored_by_outcome.png")

# Determine optimal number of clusters using silhouette score
# Test different numbers of clusters
n_clusters_range = range(3, 15)
silhouette_scores = []

for n_clusters in n_clusters_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_pca[:, :30])  # Use first 30 PCA components
    silhouette_avg = silhouette_score(X_pca[:, :30], cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"n_clusters={n_clusters}: silhouette_score={silhouette_avg:.4f}")

# Find optimal number of clusters
optimal_n = n_clusters_range[np.argmax(silhouette_scores)]
print(f"\nOptimal number of clusters (based on silhouette score): {optimal_n}")

# Plot silhouette scores
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(n_clusters_range, silhouette_scores, marker='o', linewidth=2, markersize=8)
ax.axvline(x=optimal_n, color='r', linestyle='--', label=f'Optimal: {optimal_n}')
ax.set_xlabel('Number of Clusters', fontsize=12)
ax.set_ylabel('Silhouette Score', fontsize=12)
ax.set_title('Silhouette Score vs Number of Clusters', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.savefig(output_dir / 'silhouette_scores.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: silhouette_scores.png")


In [ ]:
# Perform K-means clustering with optimal number of clusters
kmeans = KMeans(n_clusters=optimal_n, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_pca[:, :30])

case_data['cluster'] = cluster_labels

print(f"Clustering completed: {optimal_n} clusters")
print(f"Cluster sizes:")
print(case_data['cluster'].value_counts().sort_index())


In [ ]:
# Visualize clusters vs outcomes
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Plot 1: t-SNE colored by outcome
outcome_colors = {'Denied': 'red', 'Cancelled': 'orange', 'Pending': 'green', 'Other': 'gray', 'Unknown': 'black'}
for outcome in case_data['outcome'].unique():
    mask = case_data['outcome'] == outcome
    axes[0].scatter(X_tsne[mask, 0], X_tsne[mask, 1], 
                   c=outcome_colors.get(outcome, 'gray'), label=outcome, 
                   alpha=0.6, s=10, edgecolors='black', linewidth=0.1)
axes[0].set_xlabel('t-SNE Component 1', fontsize=12)
axes[0].set_ylabel('t-SNE Component 2', fontsize=12)
axes[0].set_title('t-SNE: Colored by Outcome', fontsize=14, fontweight='bold')
axes[0].legend(title='Outcome', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
axes[0].grid(True, alpha=0.3)

# Plot 2: t-SNE colored by cluster
scatter = axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=cluster_labels, 
                         cmap='tab10', s=10, alpha=0.6, edgecolors='black', linewidth=0.1)
axes[1].set_xlabel('t-SNE Component 1', fontsize=12)
axes[1].set_ylabel('t-SNE Component 2', fontsize=12)
axes[1].set_title(f't-SNE: Colored by Cluster ({optimal_n} clusters)', fontsize=14, fontweight='bold')
plt.colorbar(scatter, ax=axes[1], label='Cluster')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'tsne_clusters_vs_outcomes.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: tsne_clusters_vs_outcomes.png")


In [ ]:
# Analyze cluster-outcome relationship
# Create cross-tabulation: Cluster vs Outcome
cluster_outcome_crosstab = pd.crosstab(case_data['cluster'], case_data['outcome'], normalize='index')
cluster_outcome_counts = pd.crosstab(case_data['cluster'], case_data['outcome'])

print("Cluster-Outcome Cross-tabulation (Proportions):")
print(cluster_outcome_crosstab.round(3))
print("\nCluster-Outcome Cross-tabulation (Counts):")
print(cluster_outcome_counts)

# Calculate cluster purity for each outcome
# Purity: percentage of cases in a cluster that belong to a specific outcome
cluster_purity = {}
for outcome in ['Denied', 'Cancelled', 'Pending', 'Other']:
    cluster_purity[outcome] = cluster_outcome_crosstab[outcome].max()

print("\nCluster Purity (Maximum proportion of each outcome in any cluster):")
for outcome, purity in cluster_purity.items():
    cluster_id = cluster_outcome_crosstab[outcome].idxmax()
    print(f"  {outcome}: {purity:.4f} (Cluster {cluster_id})")

# Save cross-tabulation
cluster_outcome_crosstab.to_csv(output_dir / 'cluster_outcome_crosstab_proportions.csv')
cluster_outcome_counts.to_csv(output_dir / 'cluster_outcome_crosstab_counts.csv')
print("\nSaved: cluster_outcome_crosstab_proportions.csv")
print("Saved: cluster_outcome_crosstab_counts.csv")


In [ ]:
# Visualize cluster-outcome relationship
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# Plot 1: Heatmap of cluster vs outcome (proportions)
sns.heatmap(cluster_outcome_crosstab, annot=True, fmt='.3f', cmap='YlOrRd', 
           cbar_kws={'label': 'Proportion'}, ax=axes[0, 0])
axes[0, 0].set_xlabel('Outcome', fontsize=12)
axes[0, 0].set_ylabel('Cluster', fontsize=12)
axes[0, 0].set_title('Cluster-Outcome Association (Proportions)', fontsize=13, fontweight='bold')

# Plot 2: Stacked bar chart - Outcome distribution within each cluster
cluster_outcome_crosstab.plot(kind='bar', stacked=True, ax=axes[0, 1], 
                              colormap='Set2', width=0.8)
axes[0, 1].set_xlabel('Cluster', fontsize=12)
axes[0, 1].set_ylabel('Proportion', fontsize=12)
axes[0, 1].set_title('Outcome Distribution within Each Cluster', fontsize=13, fontweight='bold')
axes[0, 1].legend(title='Outcome', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
axes[0, 1].grid(True, alpha=0.3, axis='y')
axes[0, 1].tick_params(axis='x', rotation=0)

# Plot 3: Grouped bar chart - Cluster distribution for each outcome
outcome_cluster_crosstab = pd.crosstab(case_data['outcome'], case_data['cluster'], normalize='index')
outcome_cluster_crosstab.plot(kind='bar', ax=axes[1, 0], width=0.8, colormap='tab10')
axes[1, 0].set_xlabel('Outcome', fontsize=12)
axes[1, 0].set_ylabel('Proportion', fontsize=12)
axes[1, 0].set_title('Cluster Distribution for Each Outcome', fontsize=13, fontweight='bold')
axes[1, 0].legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
axes[1, 0].grid(True, alpha=0.3, axis='y')
axes[1, 0].tick_params(axis='x', rotation=45)

# Plot 4: Cluster sizes and dominant outcome
cluster_sizes = case_data['cluster'].value_counts().sort_index()
dominant_outcomes = cluster_outcome_crosstab.idxmax(axis=1)
cluster_summary = pd.DataFrame({
    'size': cluster_sizes,
    'dominant_outcome': dominant_outcomes
})

x_pos = np.arange(len(cluster_summary))
colors_map = {'Denied': 'red', 'Cancelled': 'orange', 'Pending': 'green', 'Other': 'gray'}
colors = [colors_map.get(outcome, 'black') for outcome in cluster_summary['dominant_outcome']]
axes[1, 1].bar(x_pos, cluster_summary['size'], color=colors, alpha=0.7, edgecolor='black')
axes[1, 1].set_xlabel('Cluster', fontsize=12)
axes[1, 1].set_ylabel('Number of Cases', fontsize=12)
axes[1, 1].set_title('Cluster Sizes (Colored by Dominant Outcome)', fontsize=13, fontweight='bold')
axes[1, 1].set_xticks(x_pos)
axes[1, 1].set_xticklabels(cluster_summary.index)
axes[1, 1].grid(True, alpha=0.3, axis='y')

# Add text labels for dominant outcome
for i, (idx, row) in enumerate(cluster_summary.iterrows()):
    axes[1, 1].text(i, row['size'] + 50, row['dominant_outcome'], 
                   ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig(output_dir / 'cluster_outcome_relationship_analysis.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: cluster_outcome_relationship_analysis.png")


In [ ]:
# Statistical analysis of cluster-outcome association
# Chi-square test to determine if cluster membership is significantly associated with outcome
chi2, p_value, dof, expected = chi2_contingency(cluster_outcome_counts)

print("=== Statistical Test: Cluster-Outcome Association ===")
print(f"Chi-square statistic: {chi2:.4f}")
print(f"p-value: {p_value:.2e}")
print(f"Degrees of freedom: {dof}")
print(f"\nInterpretation: {'Significant association' if p_value < 0.05 else 'No significant association'}")
print(f"  (p < 0.05 indicates cluster membership is significantly associated with outcome)")

# Calculate Cramér's V (measure of association strength)
n = cluster_outcome_counts.sum().sum()
cramers_v = np.sqrt(chi2 / (n * (min(cluster_outcome_counts.shape) - 1)))
print(f"\nCramér's V: {cramers_v:.4f}")
print("  Interpretation:")
if cramers_v < 0.1:
    print("    Very weak association")
elif cramers_v < 0.3:
    print("    Weak association")
elif cramers_v < 0.5:
    print("    Moderate association")
else:
    print("    Strong association")

# Identify clusters strongly associated with specific outcomes
print("\n=== Clusters Strongly Associated with Outcomes ===")
for outcome in ['Denied', 'Cancelled', 'Pending']:
    if outcome in cluster_outcome_crosstab.columns:
        # Find clusters where this outcome is dominant (>50% of cases)
        dominant_clusters = cluster_outcome_crosstab[cluster_outcome_crosstab[outcome] > 0.5]
        if len(dominant_clusters) > 0:
            print(f"\n{outcome} (>50% of cases):")
            for cluster_id, row in dominant_clusters.iterrows():
                proportion = row[outcome]
                count = cluster_outcome_counts.loc[cluster_id, outcome]
                total_in_cluster = cluster_outcome_counts.loc[cluster_id].sum()
                print(f"  Cluster {cluster_id}: {proportion:.1%} ({count}/{total_in_cluster} cases)")

# Save statistical test results
stats_results = pd.DataFrame({
    'chi2': [chi2],
    'p_value': [p_value],
    'degrees_of_freedom': [dof],
    'cramers_v': [cramers_v]
})
stats_results.to_csv(output_dir / 'cluster_outcome_statistical_test.csv', index=False)
print("\nSaved: cluster_outcome_statistical_test.csv")


In [ ]:
# Compare activity patterns across outcome groups
# Get top activities across all outcomes
all_activities_flat = []
for seq in case_data['activity_sequence']:
    all_activities_flat.extend(seq)
top_activities_all = pd.Series(all_activities_flat).value_counts().head(20).index.tolist()

# Create activity frequency matrix per outcome
activity_freq_by_outcome = []
for outcome in ['Denied', 'Cancelled', 'Pending', 'Other']:
    if outcome in case_data['outcome'].values:
        outcome_cases = case_data[case_data['outcome'] == outcome]
        all_activities_in_outcome = []
        for seq in outcome_cases['activity_sequence']:
            all_activities_in_outcome.extend(seq)
        activity_counts = pd.Series(all_activities_in_outcome).value_counts()
        total_activities = len(all_activities_in_outcome)
        
        frequencies = [activity_counts.get(activity, 0) / total_activities if total_activities > 0 else 0 
                      for activity in top_activities_all]
        activity_freq_by_outcome.append(frequencies)
    else:
        activity_freq_by_outcome.append([0] * len(top_activities_all))

activity_freq_df_outcome = pd.DataFrame(activity_freq_by_outcome, 
                                        index=['Denied', 'Cancelled', 'Pending', 'Other'],
                                        columns=top_activities_all)

# Create heatmap
fig, ax = plt.subplots(figsize=(16, 6))
sns.heatmap(activity_freq_df_outcome, annot=False, fmt='.3f', cmap='YlOrRd', 
           cbar_kws={'label': 'Activity Frequency'}, ax=ax)
ax.set_xlabel('Activity', fontsize=12)
ax.set_ylabel('Outcome', fontsize=12)
ax.set_title('Activity Pattern Heatmap by Outcome (Top 20 Activities)', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(output_dir / 'activity_patterns_by_outcome.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: activity_patterns_by_outcome.png")

# Identify activities characteristic of each outcome
print("\n=== Activities Characteristic of Each Outcome ===")
for outcome in ['Denied', 'Cancelled', 'Pending']:
    if outcome in activity_freq_df_outcome.index:
        outcome_freqs = activity_freq_df_outcome.loc[outcome]
        # Compare to overall average
        overall_avg = activity_freq_df_outcome.mean(axis=0)
        # Find activities that are more frequent in this outcome than average
        characteristic = outcome_freqs - overall_avg
        top_characteristic = characteristic.nlargest(5)
        print(f"\n{outcome} (activities more frequent than average):")
        for activity, diff in top_characteristic.items():
            freq_in_outcome = outcome_freqs[activity]
            freq_avg = overall_avg[activity]
            print(f"  {activity}: {freq_in_outcome:.4f} (avg: {freq_avg:.4f}, diff: {diff:.4f})")


In [ ]:
# Activity pattern heatmap by cluster
# Create activity frequency matrix per cluster
activity_freq_matrix = []
for cluster_id in sorted(case_data['cluster'].unique()):
    cluster_cases = case_data[case_data['cluster'] == cluster_id]
    all_activities_in_cluster = []
    for seq in cluster_cases['activity_sequence']:
        all_activities_in_cluster.extend(seq)
    activity_counts = pd.Series(all_activities_in_cluster).value_counts()
    total_activities = len(all_activities_in_cluster)
    
    frequencies = [activity_counts.get(activity, 0) / total_activities if total_activities > 0 else 0 
                  for activity in top_activities_all]
    activity_freq_matrix.append(frequencies)

activity_freq_df = pd.DataFrame(activity_freq_matrix, 
                                index=[f'Cluster {i}' for i in sorted(case_data['cluster'].unique())],
                                columns=top_activities_all)

# Annotate clusters with dominant outcome
cluster_labels_with_outcome = []
for cluster_id in sorted(case_data['cluster'].unique()):
    dominant_outcome = cluster_outcome_crosstab.loc[cluster_id].idxmax()
    prop = cluster_outcome_crosstab.loc[cluster_id, dominant_outcome]
    cluster_labels_with_outcome.append(f'Cluster {cluster_id}\n({dominant_outcome}: {prop:.1%})')

activity_freq_df.index = cluster_labels_with_outcome

# Create heatmap
fig, ax = plt.subplots(figsize=(16, 10))
sns.heatmap(activity_freq_df, annot=False, fmt='.3f', cmap='YlOrRd', 
           cbar_kws={'label': 'Activity Frequency'}, ax=ax)
ax.set_xlabel('Activity', fontsize=12)
ax.set_ylabel('Cluster (Dominant Outcome)', fontsize=12)
ax.set_title('Activity Pattern Heatmap by Cluster (Top 20 Activities)\nClusters annotated with dominant outcome', 
            fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(output_dir / 'cluster_activity_patterns_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: cluster_activity_patterns_heatmap.png")
